## ALFA data preparation part B
Global variant count data from the NCBI ALFA database was retrieved as documented in `Notebooks/Data_preparation/3-ALFA_allele_counts_a.ipynb`. This data needed to be organized and prepared in a suitable format for further analysis. The following steps were taken to achieve this:

1. Unpacking Dictionary Values: Some columns in the data contain information stored in a dictionary format. To make the data more accessible for analysis, these dictionary values were unpacked and expanded into multiple separate columns.
2. Translating Coded Population Names: The names of populations have been represented by codes in the retrieved data. To make the data easier to understand, these coded population names were translated into legible and recognizable population names.
3. Data filtering: The data was filtered, retaining only the information that will be used for downstream analysis. 
4. Formatting: To facilitate a meaningful comparison with the in-house allele count data, the retrieved global variant count data was restructured and prepared in a format that aligns with the data format used in the in-house African allele count data.

## Imports

Notebook setup

In [1]:
import os
import sys

from dotenv import load_dotenv

load_dotenv()

PROJECT_ROOT = os.getenv("PROJECT_ROOT")
if PROJECT_ROOT not in sys.path:
    os.chdir(PROJECT_ROOT + "/Notebooks")
    sys.path.append(PROJECT_ROOT)

import pandas as pd
import Utils.constants as constants
import Utils.functions as functions

Import retrieved ALFA data

In [2]:
all_alfa_allele_counts = pd.read_csv(
    os.path.join(
        PROJECT_ROOT,
        "Data",
        "Processed",
        "ALFA_allele_counts_a.csv",
    )
)

all_alfa_allele_counts.head(5)

,study_code,variant_id,population_code,reference_allele,allele_counts
0,PRJNA507278,rs552586867,SAMN10492705,C,"{'G': 0, 'C': 14050}"
1,PRJNA507278,rs552586867,SAMN10492695,C,"{'G': 0, 'C': 9690}"
2,PRJNA507278,rs552586867,SAMN10492703,C,"{'G': 0, 'C': 2898}"
3,PRJNA507278,rs552586867,SAMN10492696,C,"{'G': 0, 'C': 114}"
4,PRJNA507278,rs552586867,SAMN10492698,C,"{'G': 0, 'C': 2784}"


## Unpack dictionary values of the allele count column into multiple separate columns.

In [3]:
# Unpack dictionary of allele count column into multiple separate columns
normalised_count_columns = (
    all_alfa_allele_counts["allele_counts"].map(eval).apply(pd.Series)
)

# Drop the old allele count column from the dataframe
drop_count_column = all_alfa_allele_counts.drop(columns="allele_counts")

# Re-add the new, unpacked allele count column to the dataframe
normalised_alfa_allele_counts = pd.concat(
    [
        drop_count_column.reset_index(drop=True),
        normalised_count_columns.reset_index(drop=True),
    ],
    axis=1,
)

normalised_alfa_allele_counts.head(5)

,study_code,variant_id,population_code,reference_allele,G,C,A,T,AAAAAAAAAAA,AAAAAAAAAAAAA,...,GTGCTGCTGCTGC,GTGCTGCTGCTGCAGGTGGTGCTGCTGCTGCAGGTGGTGCTGCTGCTGC,GTGCTGCTGCTGCAGGTGGTGCTGCTGCTGC,CTCTCTCTCTC,CTCTCTCTCTCTC,CTCTCTCTCTCTCTCTC,CTCTCTCTCTCTCTCTCTC,CTCTCTCTCTCTCTCTCTCTC,CTCTCTCTCTCTCTCTCTCTCTC,CTCTCTCTCTCTCTC
0,PRJNA507278,rs552586867,SAMN10492705,C,0.0,14050.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PRJNA507278,rs552586867,SAMN10492695,C,0.0,9690.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PRJNA507278,rs552586867,SAMN10492703,C,0.0,2898.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PRJNA507278,rs552586867,SAMN10492696,C,0.0,114.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PRJNA507278,rs552586867,SAMN10492698,C,0.0,2784.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Translate population codes into legible population names.

In [4]:
# Fetch ALFA study population metadata
metadata_json = functions.get_metadata()

# Generate a list of all unique study and population codes that need to be queried
study_codes = [
    study_code for study_code in normalised_alfa_allele_counts.study_code.unique()
]
population_codes = [
    population_code
    for population_code in normalised_alfa_allele_counts.population_code.unique()
]

# Fetch study and population name metadata. Code modified from https://github.com/ncbi/dbsnp/blob/master/tutorials/Variation%20Services/Jupyter_Notebook/by_rsid.ipynb.
metadata = {}
for project_json in metadata_json:
    p = {}
    p["json"] = project_json
    p["pops"] = {}
    metadata[project_json["bioproject_id"]] = p
for prj_id, prj in metadata.items():
    functions.add_all_pops(prj["json"]["populations"], prj)

for study_id in study_codes:
    study_name = metadata[study_id]["json"]["short_name"]

    pop_dict = {
        "SAMN10492696": "African Others",
        "SAMN10492698": "African American",
        "SAMN10492697": "East Asian",
        "SAMN10492701": "Other Asian",
    }
    for pop_id in population_codes:
        if pop_id not in [
            "SAMN10492696",
            "SAMN10492698",
            "SAMN10492697",
            "SAMN10492701",
        ]:
            population_name = metadata[study_id]["pops"][pop_id]["name"]
            temp_pop_dict = {pop_id: population_name}
        pop_dict.update(temp_pop_dict)

# Add column in population allele count dataframe for population names
normalised_alfa_allele_counts["Population"] = normalised_alfa_allele_counts[
    "population_code"
].map(pop_dict)

normalised_alfa_allele_counts.head(5)

,study_code,variant_id,population_code,reference_allele,G,C,A,T,AAAAAAAAAAA,AAAAAAAAAAAAA,...,GTGCTGCTGCTGCAGGTGGTGCTGCTGCTGCAGGTGGTGCTGCTGCTGC,GTGCTGCTGCTGCAGGTGGTGCTGCTGCTGC,CTCTCTCTCTC,CTCTCTCTCTCTC,CTCTCTCTCTCTCTCTC,CTCTCTCTCTCTCTCTCTC,CTCTCTCTCTCTCTCTCTCTC,CTCTCTCTCTCTCTCTCTCTCTC,CTCTCTCTCTCTCTC,Population
0,PRJNA507278,rs552586867,SAMN10492705,C,0.0,14050.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Total
1,PRJNA507278,rs552586867,SAMN10492695,C,0.0,9690.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,European
2,PRJNA507278,rs552586867,SAMN10492703,C,0.0,2898.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,African
3,PRJNA507278,rs552586867,SAMN10492696,C,0.0,114.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,African Others
4,PRJNA507278,rs552586867,SAMN10492698,C,0.0,2784.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,African American


## Data filtering

Filter the data to include only the specific populations of interest. Retain the ALFA African Others, EUR, EAS, SAS populations.

In [5]:
filtered_alfa_allele_counts = normalised_alfa_allele_counts[
    (normalised_alfa_allele_counts["Population"] == "African Others")
    | (normalised_alfa_allele_counts["Population"] == "East Asian")
    | (normalised_alfa_allele_counts["Population"] == "South Asian")
    | (normalised_alfa_allele_counts["Population"] == "European")
]

## Restructure and prepare data in a format that aligns with the data format used in the in-house allele count data.

The data was "melted" to reorganize it in a more understandable format. In the original data, each row contained counts for multiple alleles corresponding to a single variant. To make the data easier to work with, each row was transformed so that it now represents the count for just one allele. 

In [6]:
melted_alfa_allele_counts = filtered_alfa_allele_counts.melt(
    id_vars=[
        "study_code",
        "variant_id",
        "population_code",
        "reference_allele",
        "Population",
    ],
    value_vars=filtered_alfa_allele_counts.iloc[:, 4:].columns,
    var_name="allele",
    value_name="count",
).dropna(subset="count")

melted_alfa_allele_counts.head(5)

,study_code,variant_id,population_code,reference_allele,Population,allele,count
0,PRJNA507278,rs552586867,SAMN10492695,C,European,G,0.0
1,PRJNA507278,rs552586867,SAMN10492696,C,African Others,G,0.0
2,PRJNA507278,rs552586867,SAMN10492697,C,East Asian,G,0.0
3,PRJNA507278,rs552586867,SAMN10492702,C,South Asian,G,0.0
4,PRJNA507278,rs59409892,SAMN10492695,C,European,G,0.0


In the format above, the reference allele count is not explicitly stated. To address this issue and provide a more clear representation of the data, the data was reformatted. By making this adjustment, the new data format now explicitly indicates the counts for the reference allele and the alternate allele(s) for each variant. 







In [7]:
# Separate reference allele and allele count information into different dataframes

ref_allele_info = melted_alfa_allele_counts[
    ["study_code", "variant_id", "population_code", "Population", "reference_allele"]
]
allele_count_info = melted_alfa_allele_counts[
    ["study_code", "variant_id", "population_code", "Population", "allele", "count"]
]

# Add in the allele count information corresponding to each reference allele

ref_alfa_counts = (
    ref_allele_info.merge(
        allele_count_info,
        how="left",
        left_on=[
            "study_code",
            "variant_id",
            "population_code",
            "Population",
            "reference_allele",
        ],
        right_on=[
            "study_code",
            "variant_id",
            "population_code",
            "Population",
            "allele",
        ],
    )
    .rename(columns={"count": "alfa_ref_cts"})
    .drop(columns="allele")
    .drop_duplicates(
        subset=[
            "study_code",
            "variant_id",
            "population_code",
            "Population",
            "reference_allele",
        ]
    )
)

# The remaining allele count information must be that of the alternate alleles. Add them in.

alt_alfa_counts = melted_alfa_allele_counts.merge(
    ref_alfa_counts,
    how="left",
    on=[
        "study_code",
        "variant_id",
        "population_code",
        "reference_allele",
        "Population",
    ],
).rename(columns={"allele": "alternate_allele", "count": "alfa_alt_cts"})

alfa_counts = alt_alfa_counts[
    alt_alfa_counts["reference_allele"] != alt_alfa_counts["alternate_allele"]
]

alfa_counts.head(5)

,study_code,variant_id,population_code,reference_allele,Population,alternate_allele,alfa_alt_cts,alfa_ref_cts
0,PRJNA507278,rs552586867,SAMN10492695,C,European,G,0.0,9690.0
1,PRJNA507278,rs552586867,SAMN10492696,C,African Others,G,0.0,114.0
2,PRJNA507278,rs552586867,SAMN10492697,C,East Asian,G,0.0,86.0
3,PRJNA507278,rs552586867,SAMN10492702,C,South Asian,G,0.0,98.0
4,PRJNA507278,rs59409892,SAMN10492695,C,European,G,0.0,9824.0


Restructure ALFA data to be in the same format as the in-house variant count data

In [8]:
# Pivot data
alfa_pivot_data = alfa_counts.pivot(
    index=["variant_id", "reference_allele", "alternate_allele"],
    columns="Population",
    values=["alfa_alt_cts", "alfa_ref_cts"],
)

# Separate alternate and reference count data into different dataframes to facilate renaming of count columns appropriately
alfa_data_alt = (
    alfa_pivot_data[["alfa_alt_cts"]].droplevel(level=0, axis=1).reset_index()
)
alfa_data_ref = (
    alfa_pivot_data[["alfa_ref_cts"]].droplevel(level=0, axis=1).reset_index()
)

# Add appropriate prefixes to alt and ref columns
alfa_data_alt = functions.add_prefix_dataframe_col_names(
    alfa_data_alt, alfa_data_alt.iloc[:, 3:], "ALT_CT_ALFA_"
)

alfa_data_ref = functions.add_prefix_dataframe_col_names(
    alfa_data_ref, alfa_data_ref.iloc[:, 3:], "REF_CT_ALFA_"
)

# Merge renamed alternate and reference count data
alfa_grouped_data = alfa_data_alt.merge(
    alfa_data_ref, on=["variant_id", "reference_allele", "alternate_allele"]
)

alfa_grouped_data.head(5)

Population,variant_id,reference_allele,alternate_allele,ALT_CT_ALFA_African Others,ALT_CT_ALFA_East Asian,ALT_CT_ALFA_European,ALT_CT_ALFA_South Asian,REF_CT_ALFA_African Others,REF_CT_ALFA_East Asian,REF_CT_ALFA_European,REF_CT_ALFA_South Asian
0,rs1000343,C,T,24.0,0.0,49.0,0.0,296.0,490.0,109377.0,184.0
1,rs1000989,T,C,47.0,55.0,21489.0,1685.0,135.0,109.0,37269.0,3283.0
2,rs1000990,T,C,31.0,32.0,5355.0,36.0,83.0,54.0,8931.0,62.0
3,rs1005573,C,T,111.0,35.0,10693.0,79.0,15.0,69.0,4955.0,31.0
4,rs1007311,A,G,67.0,56.0,9154.0,43.0,55.0,56.0,11242.0,55.0


## Save formatted ALFA results to a CSV

In [9]:
alfa_grouped_data.reset_index(drop=True).to_csv(
    os.path.join(
        PROJECT_ROOT,
        "Data",
        "Processed",
        "ALFA_allele_counts_b.csv",
    ),
    index=False,
)